In [ ]:
import numpy as np
import pandas as pd


# Using the average temperature of the same hour from similar days in the previous years

In [ ]:
temp_incomplete = pd.read_csv("./data/temperature.csv")

In [ ]:
def process_hour_label(hour_label):
    temp = hour_label.replace("h","")
    return int(temp)

In [ ]:
# convert h1 to number
hour_temp = temp_incomplete['hour']

hour_temp = map(process_hour_label, hour_temp)

temp_incomplete['hour'] = hour_temp

In [ ]:
temp_incomplete.head()

In [ ]:
# split incomplete data to two parts 
temp_history_complete = temp_incomplete[temp_incomplete['value'].notnull()]
temp_history_missing = temp_incomplete[temp_incomplete['value'].isnull()]

In [ ]:
# predict missing temperature values by using the average temperature values in the same hour from previous years

In [ ]:
# predict missing value by take averge 
miss_value = []

for i in xrange(temp_history_missing.shape[0]):
    row = temp_history_missing.iloc[i]
    
    temp = temp_history_complete[temp_history_complete['station_id'] == row[0]]
    temp = temp[temp['month'] == row[2]]
    temp = temp[temp['day'] == row[3]]
    temp = temp[temp['hour'] == row[4]]
    
    temp = temp['value']
    
    
    temp_sum = 0
    sum_times = 0
    for i in xrange(temp.shape[0]):
        temp_sum += temp.iloc[i]
        sum_times += 1
    
    miss_value.append( int(temp_sum/sum_times))

In [ ]:
# fill missing value 
temp_history_missing['value'] = miss_value

In [ ]:
# combine complete temperature data and missing temperature data
combination = pd.concat([temp_history_complete, temp_history_missing])

In [ ]:
# pivot-table
combination_table = pd.pivot_table(combination, values='value', index=['year', 'month', 'day', 'hour'], columns='station_id')

In [ ]:
combination_table.to_csv('./data/temp_prediction.csv', index=True)